In [ ]:
import pandas as pd

In [ ]:
data_folder = "data/raw/"
d_icd = pd.read_csv(data_folder + 'D_ICD_DIAGNOSES.csv.gz', usecols=['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']) # ICD Code lookup
diagnoses = pd.read_csv(data_folder + 'DIAGNOSES_ICD.csv.gz', usecols=['HADM_ID', 'ICD9_CODE']) # Linkage between ICD codes and Note events
drg = pd.read_csv(data_folder + 'DRGCODES.csv.gz', usecols=['HADM_ID','DESCRIPTION']) # DRG Codes
# note_events= pd.read_csv(data_folder + 'NOTEEVENTS.csv.gz', usecols=['HADM_ID','TEXT'], nrows=300000) # Number of rows = 2,083,180 without filter # We will handle this later

In [ ]:
# Data Sizes
print(f"D_ICD Shape: {d_icd.shape}")
print(f"DIAGNOSES Shape: {diagnoses.shape}")
print(f"DRG Shape: {drg.shape}")
print(f"Note Events Shape: {note_events.shape}")

In [ ]:
# Schemas
print(f"D_ICD Schema: {d_icd.columns}")
print(f"DIAGNOSES Schema: {diagnoses.columns}")
print(f"DRG Schema: {drg.columns}")
print(f"Note Events Schema: {note_events.columns}")

In [ ]:
# Explore NOTE EVENTS
data_folder = "data/raw/"
max_notes = 1
single_notes = pd.DataFrame()

for reader in pd.read_csv(data_folder + 'NOTEEVENTS.csv.gz', usecols=['HADM_ID','TEXT'], dtype={'text':str,'HADM_ID':'Int64'}, chunksize=400000):
    # Get HADM_IDs with only one note
    sub_df = reader[reader['HADM_ID'].map(reader['HADM_ID'].value_counts()) == max_notes]
    single_notes = pd.concat([single_notes, sub_df])
    sub_df.iloc[0:0]
    reader.iloc[0:0]


single_notes = single_notes[single_notes['HADM_ID'].map(single_notes['HADM_ID'].value_counts()) == max_notes]
display(single_notes.head())
print(single_notes.shape)

In [ ]:
note_events['TEXT'] = note_events['TEXT'].apply(lambda x: "\"" + str(x) + "\"")
diagnoses = diagnoses.groupby(['HADM_ID']).agg(tuple).map(list).reset_index()
joined = note_events.join(diagnoses.set_index("HADM_ID"), on=['HADM_ID'], how='inner')
# joined = joined.join(drg.set_index('HADM_ID'), on='HADM_ID', how='inner').groupby(['HADM_ID','TEXT', 'ICD9_CODE']).agg(tuple).map(list).reset_index()
# joined = joined.join(d_icd.set_index('ICD9_CODE'), on='ICD9_CODE', how='inner')

In [ ]:
print(joined.columns)
print(joined.shape)
display(joined.head(20))

In [ ]:
display(joined.loc[joined['HADM_ID'] == 100195])
display(joined.loc[joined['HADM_ID'] == 100195]['TEXT'].values[0])
display(len(joined.loc[joined['HADM_ID'] == 100195]['TEXT'].values[0]))

In [ ]:
display(d_icd.loc[d_icd['ICD9_CODE'] == '1890'])

In [ ]:
diagnoses.dtypes

In [ ]:
def remove_evm_codes(df: pd.DataFrame) -> pd.DataFrame:
    # Drop E, V, and M codes
    df['ICD9_CODE'] = df['ICD9_CODE'].apply(lambda x: str(x))
    output = df[~df['ICD9_CODE'].str.startswith("E")]
    output = output[~output['ICD9_CODE'].str.startswith("V")]   
    output = output[~output['ICD9_CODE'].str.startswith("M")]
    return output

diagnoses = remove_evm_codes(diagnoses)
print(diagnoses.head())

In [ ]:
df = diagnoses[diagnoses.icd9_code != 0]